# NoteBook utilizado para criação e avaliação dos Hiperparâmetros da árvore de decisão.

# Será inicialmente carregado com o DataSet definido no Notebook "TCC_TYAGO_AnaliseAtributos.ipynb".

Importações iniciais das bibliotecas que serão utilizadas

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
from sklearn.ensemble import GradientBoostingClassifier

Carregamento do arquivo e faz os seguintes tratamento de dados:


*   Remove registros Nulos
*   Remove regitros Inválidos
*   Substitui valores nulos por zero
*   Cria campo de diferança.   

In [ ]:
#arquivo = "/content/sample_data/TCC_Extracao_final_ni.csv"

#print("\nLendo o arquivo: {0}".format(arquivo))
    
pd_dados = pd.read_excel("/content/sample_data/TCC_Extracao_final_ni.xlsx", sheet_name=0)     
#pd_dados = pd.read_csv(arquivo, sheet_name=0) 

print(pd_dados.describe())
    
print("\nRemovendo registros nulos do arquivo")
# dropna remove os registros faltantes    
pd_dados.dropna(axis=0, subset=['sit_cadastral'], inplace = True) 
pd_dados.dropna(axis=0, subset=['cnae3'], inplace = True)
pd_dados.dropna(axis=0, subset=['ni'], inplace = True)


#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['sit_cadastral'] = pd_dados['sit_cadastral'].astype('str')
pd_dados = pd_dados[~pd_dados['sit_cadastral'].str.contains('NULA')]

print("\nRemovendo registros inválidos do arquivo")
#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['cnae3'] = pd_dados['cnae3'].astype('str')
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('IN')]
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('NI')]

print("\nSubstitui valores nulos por Zero para os campos de Débitos")
#Coloca zero quando não tem valor, para não dá erro no calculo da diferença. Quando está nulo é por que não tem mais débito, ou seja, está zerado
pd_dados[['valorm3']] = pd_dados[['valorm3']].fillna(value = 0)

print("\nCriando campos de comparação")
#calcula os campos de comparação para saber se o débito aumentou o diminuiu
pd_dados["Dif3"] = pd_dados.valorm3 - pd_dados.valor

print(pd_dados.describe())

print("\nSalvando o arquivo: {0}".format("/content/sample_data/TCC_Extracao_final_ni_Formatado.xlsx"))
pd_dados.to_excel("/content/sample_data/TCC_Extracao_final_ni_Formatado.xlsx", index=False)

                  ni         valor       valorm3
count  389285.000000  3.892850e+05  3.556940e+05
mean   191398.469052  2.506342e+04  3.055474e+04
std    111414.146668  1.813167e+06  1.658624e+06
min         0.000000  1.000000e-02  1.000000e-02
25%     94644.000000  1.958200e+02  2.035800e+02
50%    190997.000000  6.376300e+02  6.571200e+02
75%    287930.000000  1.791880e+03  1.808355e+03
max    384661.000000  9.449547e+08  4.740388e+08

Removendo registros nulos do arquivo

Removendo registros inválidos do arquivo


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Substitui valores nulos por Zero para os campos de Débitos

Criando campos de comparação
                  ni         valor       valorm3          Dif3
count  389202.000000  3.892020e+05  3.892020e+05  3.892020e+05
mean   191395.339199  2.506840e+04  2.792383e+04  2.855435e+03
std    111413.728555  1.813360e+06  1.585642e+06  1.481715e+06
min         0.000000  1.000000e-02  0.000000e+00 -4.709158e+08
25%     94639.250000  1.958025e+02  1.225800e+02  2.400000e-01
50%    190993.500000  6.376300e+02  5.632400e+02  1.790000e+00
75%    287926.750000  1.791880e+03  1.614480e+03  6.670000e+00
max    384661.000000  9.449547e+08  4.740388e+08  4.740277e+08

Salvando o arquivo: /content/sample_data/TCC_Extracao_final_ni_Formatado.xlsx


Função que define se Aumentou, Diminuiu ou Manteve

In [ ]:
def encode_units(x):
  if x < -100:
    return "Diminuiu"
  elif x > 100:
    return "Aumentou"
  else:
      return "Manteve"

# treinamento_modelo

0 - ni	
1 - classe	
2 - sit_cadastral	
3 - diferenciada	
4 - simei	
5 - simples	
6 - porte	
7 - nat_juridica	
8 - cnae3	
9 - valor	
10 - valorm3	
11 - Dif3

In [ ]:
pd_debitos = pd.read_excel("/content/sample_data/TCC_Extracao_final_ni_Formatado.xlsx", sheet_name=0) 
print(pd_debitos.keys())

#Tranforma os valores de diferença em -1 se diminuiu e 1 se aumentou o débito
pd_debitos["Dif3"] = pd_debitos["Dif3"].apply(encode_units)
#apaga os valores sem diferença, manteve o débito igual
pd_debitos.dropna(axis=0, subset=['Dif3'], inplace = True)

print("\nQuantidade de Registros depois: {0}".format(pd_debitos["classe"].count()))


var_mod =['classe','sit_cadastral', 'diferenciada', 'simei','simples', 'porte', 'nat_juridica', 'cnae3'] 
le = LabelEncoder() 
print(pd_debitos.keys())
for i in var_mod: 
  pd_debitos[i] = le.fit_transform(pd_debitos[i]) 
  #print(pd_debitos[i].sort_values().unique().tolist())
  #print(le.inverse_transform(pd_debitos[i].sort_values().unique().tolist()))

#define a coluna de resultado
y_dados_m3 = le.fit_transform(pd_debitos.iloc[:,11:12])
print(pd_debitos.iloc[:,11:12].keys())
print(le.inverse_transform([0,1,2]))
print(y_dados_m3)  

X_dados = pd_debitos.iloc[:,0:9]
#print("\nCampos: {0}".format(pd_debitos.iloc[:,0:9].keys()))
#print("\nCampos: {0}".format(X_train.head()))

debitos_tree = DecisionTreeClassifier(random_state=0) #, criterion=criterios_decisao[0], min_samples_leaf=criterios_decisao[1], min_samples_split=criterios_decisao[2], min_impurity_decrease=0.01)
#print("DecisionTreeClassifier")
debitos_tree = debitos_tree.fit(X_dados, y_dados_m3) #comando que irá treinar é o FIT
print("Acurácia", debitos_tree.score(X_dados, y_dados_m3))


for criterios_decisao in [['gini', 10000, 200000],['gini',50000, 200000]]: #['gini', 1, 2],['gini',50, 100],['gini', 50, 200], ['gini', 50, 500], ['gini', 50, 1000], ['gini', 100, 500], ['gini', 100, 1000], ['gini', 200, 500], 
    #Cria o modelo com os HyperParametros configurados, nesse caso utiliza os Defaults
    
    debitos_tree = DecisionTreeClassifier(random_state=0, criterion=criterios_decisao[0], min_samples_leaf=criterios_decisao[1], min_samples_split=criterios_decisao[2], min_impurity_decrease=0.01)
    #print("DecisionTreeClassifier")
    debitos_tree = debitos_tree.fit(X_dados, y_dados_m3) #comando que irá treinar é o FIT
    print("Acurácia", debitos_tree.score(X_dados, y_dados_m3)) #método Scrore calcula a Acurária da base de Treinamento

    Train_predict = debitos_tree.predict(X_dados) 
    print("Acurácia (previsão):", accuracy_score(y_dados_m3, Train_predict)) 
    print(classification_report(y_dados_m3, Train_predict, target_names=["Aumentou", "Diminuiu", "Manteve"]))

   
    #print("Acurancia M1: {0}, criterion: {1}, min_samples_leaf: {2}, min_samples_split: {3}".format(accuracy_score(y_dados_m3, Train_predict), debitos_tree.criterion, debitos_tree.min_samples_leaf, debitos_tree.min_samples_split))



Index(['ni', 'classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples',
       'porte', 'nat_juridica', 'cnae3', 'valor', 'valorm3', 'Dif3'],
      dtype='object')

Quantidade de Registros depois: 389202
Index(['ni', 'classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples',
       'porte', 'nat_juridica', 'cnae3', 'valor', 'valorm3', 'Dif3'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Dif3'], dtype='object')
['Aumentou' 'Diminuiu' 'Manteve']
[2 2 2 ... 1 1 2]
Acurácia 1.0
Acurácia 0.7552453481739559
Acurácia (previsão): 0.7552453481739559


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Aumentou       0.00      0.00      0.00     40776
    Diminuiu       0.00      0.00      0.00     54483
     Manteve       0.76      1.00      0.86    293943

    accuracy                           0.76    389202
   macro avg       0.25      0.33      0.29    389202
weighted avg       0.57      0.76      0.65    389202

Acurácia 0.7552453481739559
Acurácia (previsão): 0.7552453481739559


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Aumentou       0.00      0.00      0.00     40776
    Diminuiu       0.00      0.00      0.00     54483
     Manteve       0.76      1.00      0.86    293943

    accuracy                           0.76    389202
   macro avg       0.25      0.33      0.29    389202
weighted avg       0.57      0.76      0.65    389202



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd_debitos = pd.read_excel("/content/sample_data/TCC_Extracao_final_ni_Formatado.xlsx", sheet_name=0) 
print(pd_debitos.keys())

#Tranforma os valores de diferença em -1 se diminuiu e 1 se aumentou o débito
pd_debitos["Dif3"] = pd_debitos["Dif3"].apply(encode_units)
#apaga os valores sem diferença, manteve o débito igual
pd_debitos.dropna(axis=0, subset=['Dif3'], inplace = True)

print("\nQuantidade de Registros depois: {0}".format(pd_debitos["classe"].count()))


var_mod =['classe','sit_cadastral', 'diferenciada', 'simei','simples', 'porte', 'nat_juridica', 'cnae3'] 
le = LabelEncoder() 
print(pd_debitos.keys())
for i in var_mod: 
  pd_debitos[i] = le.fit_transform(pd_debitos[i]) 
  #print(pd_debitos[i].sort_values().unique().tolist())
  #print(le.inverse_transform(pd_debitos[i].sort_values().unique().tolist()))

#define a coluna de resultado
y_dados_m3 = le.fit_transform(pd_debitos.iloc[:,11:12])
print(pd_debitos.iloc[:,11:12].keys())
print(le.inverse_transform([0,1,2]))
print(y_dados_m3)  

X_dados = pd_debitos.iloc[:,0:9]
#print("\nCampos: {0}".format(pd_debitos.iloc[:,0:9].keys()))
#print("\nCampos: {0}".format(X_train.head()))

for tamanho_amostra_teste in [0.01, 0.05, 0.1, 0.15, 0.20, 0.25]:

        for criterios_decisao in [['entropy', 50, 100], ['entropy',50, 500], ['entropy', 100, 500], ['entropy', 100, 1000],['entropy', 100, 1500],['entropy', 100, 2000]]: 
        
            X_train, X_test, y_train, y_test = train_test_split(X_dados, y_dados_m3, random_state=0, test_size=tamanho_amostra_teste) #, test_size=0.1
            debitos_tree = DecisionTreeClassifier(random_state=0, criterion=criterios_decisao[0], min_samples_leaf=criterios_decisao[1], min_samples_split=criterios_decisao[2]) #, criterion="gini", min_samples_leaf=100, min_samples_split=500

            
            debitos_tree = debitos_tree.fit(X_train, y_train) #comando que irá treinar é o FIT
            #print("Acurácia (base de treinamento):", debitos_tree.score(X_train, y_train)) #método Scrore calcula a Acurária da base de Treinamento

            Train_predict = debitos_tree.predict(X_test) 
            #print("Acurácia (previsão):", accuracy_score(y_test, Train_predict)) 
            #print(classification_report(y_test, Train_predict, target_names=["Aumentou", "Diminuiu", "Manteve"]))


            y_pred = debitos_tree.predict(X_test) #tenta prever a árvore de decisão
            #print("Acurácia de previsão:", accuracy_score(y_test, y_pred))


            print("test_size: {3}, Acurancia Previsão: {4}, criterion: {0}, min_samples_leaf: {1}, min_samples_split: {2} ".format(debitos_tree.criterion, debitos_tree.min_samples_leaf, debitos_tree.min_samples_split, tamanho_amostra_teste, accuracy_score(y_test, y_pred)))
            #print("classification_report")
            #print(classification_report(y_test, y_pred, target_names=["Aumentou", "Diminuiu", "Manteve"]))
                
            #cnf_matrix = confusion_matrix(y_test, y_pred)
            #cnf_table = pd.DataFrame(data=cnf_matrix, index=["Aumentou", "Diminuiu", "Manteve"], columns=["Aumentou (prev)", "Diminuiu (prev)", "Manteve (prev)"])
            #print(cnf_table)

Index(['ni', 'classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples',
       'porte', 'nat_juridica', 'cnae3', 'valor', 'valorm3', 'Dif3'],
      dtype='object')

Quantidade de Registros depois: 389202
Index(['ni', 'classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples',
       'porte', 'nat_juridica', 'cnae3', 'valor', 'valorm3', 'Dif3'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Dif3'], dtype='object')
['Aumentou' 'Diminuiu' 'Manteve']
[2 2 2 ... 1 1 2]
test_size: 0.01, Acurancia Previsão: 0.7611096840482918, criterion: entropy, min_samples_leaf: 50, min_samples_split: 100 
test_size: 0.01, Acurancia Previsão: 0.7654764962753661, criterion: entropy, min_samples_leaf: 50, min_samples_split: 500 
test_size: 0.01, Acurancia Previsão: 0.7652196249678911, criterion: entropy, min_samples_leaf: 100, min_samples_split: 500 
test_size: 0.01, Acurancia Previsão: 0.7626509118931415, criterion: entropy, min_samples_leaf: 100, min_samples_split: 1000 
test_size: 0.01, Acurancia Previsão: 0.7629077832006165, criterion: entropy, min_samples_leaf: 100, min_samples_split: 1500 
test_size: 0.01, Acurancia Previsão: 0.7616234266632417, criterion: entropy, min_samples_leaf: 100, min_samples_split: 2000 
test_size: 0.05, Acurancia Previsão: 0.7673809156775089, criterion: entropy, min_samples_leaf: 50, min_samples_split: 100 
test_size: 0.05, Acurancia Previsão: 0.768973845